<a href="https://colab.research.google.com/github/Srivalarmathi/Techjays_Inventory_Optimization_VM/blob/main/notebooks/Techjays_Inventory_Optimization_VM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Techjays Inventory Optimization
Author: **Valarmathi Ganessin**

Environment: **Google Colab**

Project: **Predictive Inventory Management System**

 # 1. Executive Summary

This notebook builds a predictive inventory management system for Techjays.  
It forecasts product‑level demand, calculates optimal stock levels, and identifies stockout/overstock risks.

Key outputs:
- Cleaned and unified dataset
- Demand forecasts per product
- Safety Stock, Reorder Point (ROP), EOQ
- Inventory risk classification
- Actionable business recommendations


## 2. Business Problem & Objectives

### Business Problem
Techjays needs a data‑driven system to:
- Predict future sales
- Prevent stockouts
- Reduce overstock
- Improve inventory turnover

### Objectives
1. Forecast demand per product
2. Recommend optimal stock levels
3. Identify inventory risks
4. Provide actionable insights for purchasing and planning


## 3. Data Loading & Preprocessing

### Steps:
- Load all 7 sheets from Excel
- Convert date columns
- Apply business cleaning rules:
  - Keep data from 2019+
  - Remove C*/Q* locations
  - Remove invalid product codes
  - Remove products with no sales in 2024–2025
  - Remove negative inventory quantities
  - Aggregate quantities per product per date
  - Keep products with ≥ 5 data points
- Create unified analysis tables


##  TASK 1 — Data Cleaning Summary Table (VM PRO‑TOUCH)

This table summarizes the cleaning rules required in the assignment and the additional professional enhancements applied to prepare the dataset for forecasting and inventory optimization.

### Table 1: Assignment‑Required Cleaning vs VM Pro‑Touch Enhancements

| **Cleaning Step** | **Requested in Assignment?** | **Purpose (Business Reason)** | **VM Pro‑Touch Additions?** | **Why It Matters for Forecasting & Inventory Modeling** |
|-------------------|------------------------------|-------------------------------|------------------------------|----------------------------------------------------------|
| Load all 7 sheets | ✔ Yes | Bring all data sources into the environment | — | Required to build unified dataset |
| Convert date columns | ✔ Yes | Ensure proper time‑series sorting & filtering | — | Forecasting models require valid datetime format |
| Keep data from 2019+ | ✔ Yes | Remove outdated patterns; focus on recent demand | — | Old data distorts trends & seasonality |
| Remove C*/Q* locations | ✔ Yes | Exclude consignment & quarantine stock | — | These locations do not represent usable inventory |
| Remove invalid product codes | ✔ Yes | Remove service items, freight, processing codes | — | These are not physical products → cannot forecast |
| Remove products with no sales in 2024–2025 | ✔ Yes | Focus only on active products | — | Forecasting dead stock is meaningless |
| Remove negative inventory quantities | ✔ Yes | Fix data entry errors | — | Negative stock breaks ROP & safety stock formulas |
| Aggregate sales per product per date | ✔ Yes | Create clean daily demand | — | Time‑series requires one value per day |
| Keep products with ≥ 5 data points | ✔ Yes | Ensure minimum history for forecasting | — | Models cannot learn patterns with too few points |
| Merge product attributes (lead time, group) | ✖ Not explicitly asked | Add product metadata for ROP & category analysis | ✔ Added | Lead time is required for ROP; group helps EDA |
| Merge product group codes | ✖ Not asked | Add category-level insights | ✔ Added | Helps identify patterns by product family |
| Remove retired products | ✖ Not asked | Exclude discontinued items | ✔ Added | Prevents forecasting obsolete SKUs |
| Remove products with zero total sales | ✖ Not asked | Remove irrelevant SKUs | ✔ Added | Zero-demand SKUs distort model accuracy |
| Remove products with no inventory ever | ✖ Not asked | Ensure product exists in stock | ✔ Added | Needed for inventory optimization |
| Remove duplicates | ✖ Not asked | Fix data quality issues | ✔ Added | Prevents double-counting demand |
| Standardize product codes (strip, uppercase) | ✖ Not asked | Fix inconsistent ProdCode formatting | ✔ Added | Ensures correct merging across sheets |
| Validate missing values | ✖ Not asked | Ensure no critical fields are null | ✔ Added | Missing dates or ProdCodes break models |
| Validate data types | ✖ Not asked | Ensure numeric fields are numeric | ✔ Added | Required for calculations (ROP, EOQ, etc.) |
| Create unified analysis tables | ✔ Yes | Combine cleaned data for modeling | ✔ Enhanced | Produces final modeling-ready dataset |

---

### Table 2: Mandatory Cleaning for Forecasting & Inventory Optimization

| **Mandatory for Forecasting?** | **Mandatory for Inventory Optimization?** | **Cleaning Step** | **Why Mandatory** |
|-------------------------------|-------------------------------------------|--------------------|--------------------|
| ✔ Yes | ✔ Yes | Convert dates | Time‑series cannot run without valid dates |
| ✔ Yes | ✔ Yes | Aggregate sales per day | Forecasting requires one value per day |
| ✔ Yes | ✔ Yes | Remove invalid product codes | Non‑products break forecasting & ROP |
| ✔ Yes | ✔ Yes | Remove negative inventory | ROP & safety stock formulas fail |
| ✔ Yes | ✔ Yes | Keep products with ≥ 5 data points | Models need minimum history |
| ✔ Yes | ✔ Yes | Remove products with no recent sales | Forecasting dead stock is meaningless |
| ✔ Yes | ✔ Yes | Merge lead time | ROP = Avg Daily Demand × Lead Time + Safety Stock |
| ✔ Yes | ✔ Yes | Standardize product codes | Ensures correct joins |
| ✔ Yes | ✖ No | Remove retired products | No need to forecast discontinued items |
| ✖ No | ✔ Yes | Remove consignment/quarantine | These are not usable inventory |
| ✖ No | ✔ Yes | Remove products with no inventory | Cannot calculate ROP/EOQ without stock |
| ✔ Yes | ✔ Yes | Validate missing values | Missing ProdCode/Date breaks models |

---

### Table 3: Why These Cleaning Steps Matter (Business Impact)

| **Cleaning Step** | **Impact on Forecasting** | **Impact on Inventory Optimization** |
|-------------------|---------------------------|--------------------------------------|
| Remove invalid product codes | Prevents noise in demand | Prevents wrong ROP calculations |
| Remove old data | Improves trend accuracy | Reflects current stocking needs |
| Remove negative inventory | Prevents model errors | Prevents negative ROP |
| Aggregate sales | Creates stable time series | Enables daily demand calculation |
| Merge lead time | Needed for forecasting horizon | Required for ROP formula |
| Remove dead stock | Avoids flat zero forecasts | Avoids unnecessary stock holding |
| Standardize ProdCode | Ensures correct joins | Ensures correct inventory mapping |


In [3]:
# Code: mount drive, set file path, load sheets
# ------------------------------------------------------------
# DATA LOADING SECTION
# ------------------------------------------------------------
# This block uploads the Excel file from your laptop and loads
# all required sheets into pandas DataFrames.
# Each sheet contains two non‑data rows (title + blank), so
# skiprows=2 ensures the actual column headers are used.
# ------------------------------------------------------------

from google.colab import files
uploaded = files.upload()

import pandas as pd

# Use the exact filename you uploaded
file_path = "Techjays_Inventory.xlsx"

# Load each sheet with skiprows=2 to remove title rows
inventory_df = pd.read_excel(file_path, sheet_name="Inventory", skiprows=2)
sales_df = pd.read_excel(file_path, sheet_name="Sales", skiprows=2)
receiving_df = pd.read_excel(file_path, sheet_name="Receiving Transactions", skiprows=2)
products_df = pd.read_excel(file_path, sheet_name="Products", skiprows=2)
comp_df = pd.read_excel(file_path, sheet_name="Comp", skiprows=2)
product_group_df = pd.read_excel(file_path, sheet_name="Product Group", skiprows=2)
locations_df = pd.read_excel(file_path, sheet_name="Locations", skiprows=2)



Saving Techjays_Inventory.xlsx to Techjays_Inventory.xlsx


In [4]:
# Code: inspect shapes, missing values, data types
# ------------------------------------------------------------
# STEP 2: Inspect shapes, missing values, and data types
# ------------------------------------------------------------
# This section helps verify the structure of each dataset.
# We check:
#   1. Shape  – number of rows and columns
#   2. Missing values – to identify gaps in the data
#   3. Data types – to ensure columns are correctly typed
# ------------------------------------------------------------

datasets = {
    "inventory": inventory_df,
    "sales": sales_df,
    "receiving": receiving_df,
    "products": products_df,
    "comp": comp_df,
    "product_group": product_group_df,
    "locations": locations_df,
}

for name, df in datasets.items():
    print("------------------------------------------------------------")
    print(f"Dataset: {name}")
    print("Shape:", df.shape)
    print("\nMissing values:")
    print(df.isnull().sum())
    print("\nData types:")
    print(df.dtypes)
    print("------------------------------------------------------------\n")



------------------------------------------------------------
Dataset: inventory
Shape: (11369, 28)

Missing values:
InventoryDtlID               0
HeatID                     512
ProdCode                     1
Qty                         15
Heat                        46
Lot                       3217
Vendor                    9677
VendorID                    34
Cost                        23
PONum                       92
Date                        27
Comments                  1475
Location                    24
CustRet                      0
CustID                   10209
Hold                         0
ysnConsignment               0
lngTag                    2230
dblQtyRecd                1413
dblOrgCost                1756
lngVendorRatingID         3672
lngPODetailID             4032
lngRecID                  4032
dblCOG                       0
dblFreight                   0
dtmMaturationDate         9708
ysnOffConsignment            0
dtmDateOffConsignment     9873
dtype: int64

Da

# **Missing Value Summary Tables**

### Inventory – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| ProdCode | 1 | Yes | Drop row | Cannot fill |
| Qty | 15 | Yes | Drop row | Cannot fill |
| Date | 27 | Yes | Drop row | Cannot fill |
| Location | 24 | Yes | Drop row | Cannot fill |
| Cost | 23 | No | Ignore | Not needed |
| Heat, Lot, Vendor, VendorID, Comments, CustID, etc. | Many | No | Ignore | Not needed |
| dtmMaturationDate, dtmDateOffConsignment | Many | No | Ignore | Not needed |


### Sales – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| QtyShipped | 5 | Yes | Drop row | Cannot fill |
| Price | 18 | No | Ignore | Not needed |
| Cost | 80 | No | Ignore | Not needed |
| TotSale | 22 | No | Ignore | Not needed |
| TotCost | 85 | No | Ignore | Not needed |
| Profit | 101 | No | Ignore | Can be recalculated |
| %MU | 783 | No | Ignore | Not needed |
| DueDate | 11 | No | Ignore | Not needed |


### Receiving – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| Qty | 1 | Yes | Drop row | Cannot fill |
| ProdCode | 0 | Yes | Keep | — |
| Date | 0 | Yes | Keep | — |
| PONum | 7 | No | Ignore | Not needed |
| VendorID | 39 | No | Ignore | Not needed |
| HeatID, Lot, Comments, InvoiceNum | Many | No | Ignore | Not needed |


### Products – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| ProdCode | 0 | Yes | Keep | — |
| ProductGroup | 754 | Optional | Ignore | Not needed |
| Unit | 1 | No | Ignore | Not needed |
| dblWtPerInch | 349 | No | Ignore | Not needed |
| dblPCLength | 313 | No | Ignore | Not needed |
| intLeadTime | 674 | No | Ignore | Not needed |
| VendorDesc, Desc2, StockID, AWSClass | Many | No | Ignore | Not needed |


### Comp – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| Company | 227 | No | Ignore | Not needed |
| CustNum | 905 | No | Ignore | Not needed |
| CITY, STATE | 255–264 | No | Ignore | Not needed |
| All other admin fields | Many | No | Ignore | Not needed |


### Product Group – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| GroupID | 0 | Yes | Keep | — |
| Description | 139 | No | Ignore | Not needed |
| strProductGroupCode | 3 | Optional | Ignore | Not needed |


### Locations – Missing Value Handling

| Column | Missing Count | Required for Forecasting | Recommended Action | Best Filling Method |
|--------|----------------|--------------------------|---------------------|----------------------|
| All columns | 0 | Yes | Keep | — |


In [5]:
# ------------------------------------------------------------
# STEP 3.1 — Convert date columns to proper datetime format
# ------------------------------------------------------------
# Why:
#   - Excel sometimes stores dates as strings or mixed formats.
#   - Forecasting and filtering require valid datetime objects.
#   - errors="coerce" converts invalid dates to NaT (safe handling).
# ------------------------------------------------------------
sales_df["DATE"] = pd.to_datetime(sales_df["DATE"], errors="coerce")
inventory_df["Date"] = pd.to_datetime(inventory_df["Date"], errors="coerce")
receiving_df["Date"] = pd.to_datetime(receiving_df["Date"], errors="coerce")

print("Sales after date conversion:", sales_df.shape)
display(sales_df.head(5))

print("Inventory after date conversion:", inventory_df.shape)
display(inventory_df.head(5))

print("Receiving after date conversion:", receiving_df.shape)
display(receiving_df.head(5))



In [6]:

print("Sales after date conversion:", sales_df.shape)
display(sales_df.head(5))

print("Inventory after date conversion:", inventory_df.shape)
display(inventory_df.head(5))

print("Receiving after date conversion:", receiving_df.shape)
display(receiving_df.head(5))


Sales after date conversion: (17413, 14)


,CompID,SalesPerson,InvoiceNum,DATE,Company,ProdCode,QtyShipped,Price,Cost,TotSale,TotCost,Profit,%MU,DueDate
0,1384,ADRIAN GARCIA,73063,2025-06-19,SPECIALTY MATERIALS,4.500718RB,58.0,51.60,41.22983,2992.80,2391.33014,601.46986,0.251521,2025-07-19
1,1384,ADRIAN GARCIA,73065,2025-06-19,SPECIALTY MATERIALS,1.125405HB,150.0,25.67,21.16000,3850.50,3174.00000,676.50000,0.213138,2025-07-19
2,1384,ADRIAN GARCIA,73066,2025-06-19,SPECIALTY MATERIALS,2.000718RB,11.0,38.07,20.14000,418.77,221.54000,197.23000,0.890268,2025-07-19
3,1384,ADRIAN GARCIA,73480,2025-10-30,SPECIALTY MATERIALS,0.750174RB,36.0,9.16,2.81000,329.76,101.16000,228.60000,2.259786,NaT
4,122,CLAYTON BAILYE,67141,2020-02-10,VEGAS FASTENER MANUFACTURING,1.500K500RBGF,0.0,13.78,11.85000,0.00,0.00000,0.00000,NaN,2020-03-11


Inventory after date conversion: (11369, 28)


,InventoryDtlID,HeatID,ProdCode,Qty,Heat,Lot,Vendor,VendorID,Cost,PONum,...,dblQtyRecd,dblOrgCost,lngVendorRatingID,lngPODetailID,lngRecID,dblCOG,dblFreight,dtmMaturationDate,ysnOffConsignment,dtmDateOffConsignment
0,28001,0.0,0.9375405HB,2280.0,MM03W7RR17,NaN,NaN,224.0,21.31,3328.0,...,2280.0,0.0,3522.0,4705.0,9975.0,21.31,0.0,NaT,False,NaT
1,28000,0.0,1.250K500RBGFAH,770.0,MM03W3KG14,NaN,NaN,224.0,17.68,3086.0,...,770.0,0.0,3521.0,3995.0,9974.0,17.68,0.0,NaT,False,NaT
2,27997,0.0,1.625405HB,393.0,5G1001-06,W97311,NaN,145.0,17.68,3653.0,...,393.0,0.0,3519.0,5780.0,9973.0,17.68,0.0,NaT,False,NaT
3,27996,0.0,1.625405HB,500.0,5G1001-06,W97311,NaN,145.0,17.68,3653.0,...,500.0,0.0,3519.0,5780.0,9972.0,17.68,0.0,NaT,False,NaT
4,27995,0.0,1.625405HB,513.0,5G0998-04,W97311,NaN,145.0,17.68,3653.0,...,513.0,0.0,3519.0,5780.0,9971.0,17.68,0.0,NaT,False,NaT


Receiving after date conversion: (9567, 23)


,RecID,Date,InvNum,PONum,VendorID,ProdCode,HeatID,Qty,Heat,Lot,...,Post,Loc,CustID,InvoiceNum,ysnConsignment,lngTag,lngPODetailID,dblFreight,intMaturationDays,dtmMaturationDate
0,9952,2025-12-15,NaN,3725.0,465.0,MISC,NaN,1.0,NaN,NaN,...,True,STL,0,NaN,False,10326.0,5961.0,NaN,NaN,NaT
1,9951,2025-12-15,NaN,3725.0,465.0,MISC,NaN,1.0,NaN,NaN,...,True,STL,0,NaN,False,10325.0,5960.0,NaN,NaN,NaT
2,9928,2025-12-11,NaN,3723.0,465.0,MISC,NaN,1.0,NaN,NaN,...,True,STL,0,NaN,False,10302.0,5957.0,NaN,NaN,NaT
3,9966,2025-12-18,NaN,3722.0,175.0,11.000304RB,NaN,4135.0,183441,6427230,...,True,HOU,0,NaN,False,10340.0,5938.0,0.0,NaN,NaT
4,9957,2025-12-18,NaN,3722.0,175.0,3.500304RB,NaN,18.0,182929,6419880,...,True,HOU,0,NaN,False,10331.0,5937.0,0.0,NaN,NaT


In [7]:
# Code: apply all cleaning rules
# ------------------------------------------------------------
# STEP 3.3 — Remove locations starting with 'C' or 'Q'
# ------------------------------------------------------------
# Why:
#   - 'C' = Consignment, 'Q' = Quarantine.
#   - These locations do not represent usable inventory.
#   - Must be excluded for accurate stock and demand modeling.
# ------------------------------------------------------------

inventory_df = inventory_df[~inventory_df["Location"].astype(str).str.startswith(("C", "Q"))]
receiving_df = receiving_df[~receiving_df["Loc"].astype(str).str.startswith(("C", "Q"))]

print("Inventory after removing C*/Q* locations:", inventory_df.shape)
display(inventory_df.head(5))

print("Receiving after removing C*/Q* locations:", receiving_df.shape)
display(receiving_df.head(5))


Inventory after removing C*/Q* locations: (10167, 28)


,InventoryDtlID,HeatID,ProdCode,Qty,Heat,Lot,Vendor,VendorID,Cost,PONum,...,dblQtyRecd,dblOrgCost,lngVendorRatingID,lngPODetailID,lngRecID,dblCOG,dblFreight,dtmMaturationDate,ysnOffConsignment,dtmDateOffConsignment
0,28001,0.0,0.9375405HB,2280.0,MM03W7RR17,NaN,NaN,224.0,21.31,3328.0,...,2280.0,0.0,3522.0,4705.0,9975.0,21.31,0.0,NaT,False,NaT
1,28000,0.0,1.250K500RBGFAH,770.0,MM03W3KG14,NaN,NaN,224.0,17.68,3086.0,...,770.0,0.0,3521.0,3995.0,9974.0,17.68,0.0,NaT,False,NaT
2,27997,0.0,1.625405HB,393.0,5G1001-06,W97311,NaN,145.0,17.68,3653.0,...,393.0,0.0,3519.0,5780.0,9973.0,17.68,0.0,NaT,False,NaT
3,27996,0.0,1.625405HB,500.0,5G1001-06,W97311,NaN,145.0,17.68,3653.0,...,500.0,0.0,3519.0,5780.0,9972.0,17.68,0.0,NaT,False,NaT
4,27995,0.0,1.625405HB,513.0,5G0998-04,W97311,NaN,145.0,17.68,3653.0,...,513.0,0.0,3519.0,5780.0,9971.0,17.68,0.0,NaT,False,NaT


Receiving after removing C*/Q* locations: (9526, 23)


,RecID,Date,InvNum,PONum,VendorID,ProdCode,HeatID,Qty,Heat,Lot,...,Post,Loc,CustID,InvoiceNum,ysnConsignment,lngTag,lngPODetailID,dblFreight,intMaturationDays,dtmMaturationDate
0,9952,2025-12-15,NaN,3725.0,465.0,MISC,NaN,1.0,NaN,NaN,...,True,STL,0,NaN,False,10326.0,5961.0,NaN,NaN,NaT
1,9951,2025-12-15,NaN,3725.0,465.0,MISC,NaN,1.0,NaN,NaN,...,True,STL,0,NaN,False,10325.0,5960.0,NaN,NaN,NaT
2,9928,2025-12-11,NaN,3723.0,465.0,MISC,NaN,1.0,NaN,NaN,...,True,STL,0,NaN,False,10302.0,5957.0,NaN,NaN,NaT
3,9966,2025-12-18,NaN,3722.0,175.0,11.000304RB,NaN,4135.0,183441,6427230,...,True,HOU,0,NaN,False,10340.0,5938.0,0.0,NaN,NaT
4,9957,2025-12-18,NaN,3722.0,175.0,3.500304RB,NaN,18.0,182929,6419880,...,True,HOU,0,NaN,False,10331.0,5937.0,0.0,NaN,NaT


In [ ]:
# Code: create sales_daily and sales_monthly tables


## 4. Exploratory Data Analysis (EDA)

### Goals:
- Understand demand patterns
- Identify fast/slow movers
- Analyze sales by product group, location, customer
- Compare receiving vs sales
- Compute basic inventory KPIs


In [ ]:
# Code: overall monthly sales trend


In [ ]:
# Code: top products by volume


In [ ]:
# Code: velocity classification (fast vs slow movers)


In [ ]:
# Code: sales by product group / location / customer


### Key Insights
- Bullet points summarizing findings


## 5. Forecasting Demand

### Approach:
- Prepare time series per product
- Use baseline model (moving average / exponential smoothing)
- Optional: ARIMA / Prophet for selected SKUs
- Evaluate using RMSE, MAE, MAPE


In [ ]:
# Code: prepare time series (monthly)


In [ ]:
# Code: baseline forecasting function


In [ ]:
# Code: evaluate model performance


In [ ]:
# Code: plot historical vs forecast for sample products


## 6. Inventory Optimization

### Metrics:
- Average Daily Demand
- Standard Deviation of Demand
- Safety Stock
- Reorder Point (ROP)
- EOQ (with assumptions)

### Formulas:
Safety Stock = Z × StdDev × √LeadTime  
ROP = (Avg Daily Demand × Lead Time) + Safety Stock  
EOQ = √(2DS / H)


In [ ]:
# Code: compute avg daily demand & std dev


In [ ]:
# Code: compute safety stock


In [ ]:
# Code: compute reorder point (ROP)


In [ ]:
# Code: compute EOQ (optional)



In [ ]:
# Code: final recommendation table


## 7. Dashboard‑Style Visualizations

- Current vs recommended inventory
- Stockout risk items
- Overstocked items
- Forecast charts for key products


In [ ]:
# Code: bar chart – current vs recommended stock


In [ ]:
# Code: table – stockout risk items


In [ ]:
# Code: table – overstocked items


## 8. Final Recommendations

### Business Insights
- Bullet points summarizing key findings

### Operational Recommendations
- Increase stock for…
- Reduce stock for…
- Improve lead time for…

### Limitations
- Data gaps
- Missing cost parameters
- Forecast uncertainty

### Next Steps
- Deploy automated pipeline
- Build Power BI dashboard
- Add external features (seasonality, pricing, etc.)


## 9. Appendix

- Helper functions
- Additional plots
- Data dictionary
